In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import torch.nn as nn
from layers.customLayer import CustomLayer
from training.train import TrainUtil
from metrics.charts import ChartUtil
from metrics.charts import TrainingCharts
from datasets.xor import XOR
from experiment.runExperiment import RunExperiment
from experiment.models import ModelSetups

In [6]:
dataset = XOR()
in_features = 4
out_features = 2
X_train, y_train, X_val, y_val = dataset.get_dataset(in_features = in_features, unique=False)
trainUtil = TrainUtil(X_train, y_train, X_val = X_val, y_val = y_val)

In [ ]:
experiment = RunExperiment(trainUtil, in_features, ModelSetups().get_three_layer_models)

In [ ]:
dataset = XOR()
X_train, y_train, X_val, y_val = dataset.get_dataset(in_features = 6, unique = True)
trainUtilSixFeatures = TrainUtil(X_train, y_train, X_val = X_val, y_val = y_val)
class LargerSparseModel(nn.Module):
    def __init__(self, in_features = 6, out_features = 2):
        super().__init__()
        self._layer1 = CustomLayer(in_features, 
                                        400, 
                                        out_features_allowed=50)
        self._layer2 = CustomLayer(400, 500,out_features_allowed=100)
        self._layer3 = CustomLayer(500, 1000,out_features_allowed=200)
        self._layer4 = CustomLayer(1000, 400,out_features_allowed=100)
        self._layer5 = CustomLayer(400, out_features)

    def forward(self, x, return_acts = False):
        x_l1 = self._layer1(x)
        x_l2 = self._layer2(x_l1)
        x_l3 = self._layer3(x_l2)
        x_l4 = self._layer4(x_l3)
        out = self._layer5(x_l4)
        if return_acts:
            return out, {"l1": x_l1, "l2": x_l2, "l3": x_l3, "l4": out}
        return out
chartUtilsSixFeatures = ChartUtil()
for _ in range(50):
    loss_over_epochs, accuracy_over_epochs, epoch_numbers, train_accuracy, val_accuracy = trainUtilSixFeatures.train_and_evaluate(LargerSparseModel(), 
                                                                                                                           epochs=1000, 
                                                                                                                           name="Sparse Model")
    chartUtilsSixFeatures.add_train_data("Sparse Model", loss_over_epochs, accuracy_over_epochs, epoch_numbers)
    chartUtilsSixFeatures.add_test_data("Sparse Model", train_accuracy, val_accuracy)

In [ ]:
chartUtilsSixFeaturesBatch = ChartUtil()
for _ in range(50):
    loss_over_epochs, accuracy_over_epochs, epoch_numbers, train_accuracy, val_accuracy = trainUtilSixFeatures.train_and_evaluate(LargerSparseModel(), 
                                                                                                                           epochs=200, 
                                                                                                                           name="Sparse Model",
                                                                                                                           batch_train=True)
    chartUtilsSixFeaturesBatch.add_train_data("Sparse Model", loss_over_epochs, accuracy_over_epochs, epoch_numbers)
    chartUtilsSixFeaturesBatch.add_test_data("Sparse Model", train_accuracy, val_accuracy)

In [ ]:
chartUtilsSixFeaturesBatch.plot_training_data_for().show()
chartUtilsSixFeaturesBatch.plot_training_data_for(TrainingCharts.ACC, height_alpha=1).show()
chartUtilsSixFeaturesBatch.plot_test_accu_for_models(width_alpha=2.5).show()

In [ ]:
in_features = [2, 4, 6, 8]
trainUtils = []
chartUtilsBatch = []
for i in in_features:
    dataset = XOR()
    X_train, y_train, X_val, y_val = dataset.get_dataset(in_features = i, unique = True)
    newTrainUtil = TrainUtil(X_train, y_train, X_val = X_val, y_val = y_val)
    trainUtils.append(newTrainUtil)
    chartUtil = experiment.perform_one_experiment(n_epochs= 500, trainUtil=newTrainUtil, in_features = i, batch_train = True)
    chartUtilsBatch.append(chartUtil)